In [10]:
import sys
import os
import torch

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='2'

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch.nn.functional as F

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from equivariant_pose_graph.utils.color_utils import color_gradient

from pytorch3d.transforms import Transform3d, Rotate, Translate

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def toDisplay(x, target_dim = None):
    while(target_dim is not None and x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [12]:
from pytorch3d.transforms import Transform3d,Rotate, rotation_6d_to_matrix, se3_exp_map
from equivariant_pose_graph.utils.se3 import *
from equivariant_pose_graph.utils.display import *
from equivariant_pose_graph.dataset.point_cloud_dataset import PointCloudDataset

In [17]:
from equivariant_pose_graph.models.transformer_flow import ResidualFlow_DiffEmbTransformer
from equivariant_pose_graph.models.multimodal_transformer_flow import Multimodal_ResidualFlow_DiffEmbTransformer, Multimodal_ResidualFlow_DiffEmbTransformer_WithPZCondX
from equivariant_pose_graph.training.flow_equivariance_training_module_nocentering_eval_init import EquivarianceTestingModule
from equivariant_pose_graph.training.flow_equivariance_training_module_nocentering_multimodal import EquivarianceTrainingModule, EquivarianceTrainingModule_WithPZCondX

# place_checkpoint_file='/home/chuerp/good_mug_models/rm/epoch=120-step=15125.ckpt'
# place_network = ResidualFlow_DiffEmbTransformer(
#                     emb_nn='dgcnn', return_flow_component=True, center_feature=True,
#                     inital_sampling_ratio=1)
# place_model = EquivarianceTestingModule(
#     place_network,
#     lr=1e-4,
#     image_log_period=100,
#     weight_normalize='l1',
#     loop=1
# )

# noreg
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-11-21_154442/residual_flow_occlusion/2fk29m3e/checkpoints/epoch_27_global_step_7000.ckpt"
# noreg continued
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-12-01_025300/residual_flow_occlusion/1db7s7cq/checkpoints/epoch=125-step=31500.ckpt"
# uniform reg
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-11-30_155029/residual_flow_occlusion/17bt6tod/checkpoints/epoch_43_global_step_11000.ckpt"
# noreg overall mean center
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-12-01_025300/residual_flow_occlusion/1db7s7cq/checkpoints/epoch_123_global_step_31000.ckpt"
# noreg overall mean center. exp dense point cloud
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-12-01_234630/residual_flow_occlusion/2ghi50y6/checkpoints/epoch_103_global_step_26000.ckpt"
# L2 norm conditioning
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-12-20_175830/residual_flow_occlusion/3ppf0g8p/checkpoints/epoch_111_global_step_28000.ckpt"
# ^ l2 norm conditiong but With p(z|X)
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-12-22_002311/residual_flow_occlusion/15zfvyx1/checkpoints/epoch_75_global_step_19000.ckpt"
# ^ but overfit on the overfit train dataset
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2022-12-23_014459/residual_flow_occlusion/1e4vc8gx/checkpoints/epoch_35_global_step_9000.ckpt"
# ^ but the loss is only the p(z|X) loss, so it acutally optimizes
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-04_235552/residual_flow_occlusion/wcshkbzy/checkpoints/epoch_35_global_step_9000.ckpt"
# [overfit dataset] low lr and has bad kl loss. however, this looks like it has a better latent p(z|X)?
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-04_121156/residual_flow_occlusion/10mhwx6g/checkpoints/epoch_199_global_step_50000.ckpt"
# mse loss?
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-05_003452/residual_flow_occlusion/qhfhic8n/checkpoints/epoch_7_global_step_2000.ckpt"
# mse loss with only one dgcnn (no anchor/action separation)
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-05_010323/residual_flow_occlusion/3dhhdrih/checkpoints/epoch_11_global_step_3000.ckpt"
# mse loss trained on 2 mugs in the train dataset
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-05_223316/residual_flow_occlusion/2vwec0gm/checkpoints/epoch_35_global_step_9000.ckpt"
# mse loss trained on 3 mugs
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-05_231713/residual_flow_occlusion/103sexr1/checkpoints/epoch_39_global_step_10000.ckpt"
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-06_000501/residual_flow_occlusion/sp1sof97/checkpoints/epoch_75_global_step_19000.ckpt"
# inverse kl + all mugs
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-06_090446/residual_flow_occlusion/1hjwr3f8/checkpoints/epoch_15_global_step_4000.ckpt"
# forward kl + mse + all mugs
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-06_090351/residual_flow_occlusion/uqaun9iz/checkpoints/epoch_15_global_step_4000.ckpt"
# normed inverse kl + all mugs + 2 DGCNN
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-09_162211/residual_flow_occlusion/2i7yifsv/checkpoints/epoch_143_global_step_36000.ckpt"
# normed inverse kl + all mugs + 1 DGCNN not 2
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-10_000911/residual_flow_occlusion/1ds271fl/checkpoints/epoch_159_global_step_40000.ckpt"
# normed forward kl + all mugs + 1 DGCNN not 2
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-10_195720/residual_flow_occlusion/z922svt3/checkpoints/epoch_23_global_step_6000.ckpt"
# forward kl + starting from pretrained inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-11_165435/residual_flow_occlusion/5kdqbhid/checkpoints/epoch_99_global_step_25000.ckpt"
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-11_165435/residual_flow_occlusion/5kdqbhid/checkpoints/epoch_11_global_step_3000.ckpt"
# 
# inverse kl but 2D probabilities for the 2 objects
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_122937/residual_flow_occlusion/22dsk7ml/checkpoints/epoch_71_global_step_18000.ckpt"

# inverse kl but with kl(mug) + kl(rack) loss
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_001023/residual_flow_occlusion/3czoym0x/checkpoints/epoch_95_global_step_24000.ckpt"

# mug only inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_130619/residual_flow_occlusion/1mmwz1ae/checkpoints/epoch_31_global_step_8000.ckpt"


# 1 DGCNN
# 1 mug overfit inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_150034/residual_flow_occlusion/3k1e3uq6/checkpoints/epoch_11_global_step_3000.ckpt"
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_152242/residual_flow_occlusion/3pgp6833/checkpoints/epoch_3_global_step_1000.ckpt"
# 2 mug overfit inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_155828/residual_flow_occlusion/2nswo2uu/checkpoints/epoch_35_global_step_9000.ckpt"
# 3 mug overfit inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-12_155847/residual_flow_occlusion/1rbhvbfr/checkpoints/epoch_11_global_step_3000.ckpt"

# 2 DGCNN
# 1 mug overfit inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-17_145122/residual_flow_occlusion/23n57sp0/checkpoints/epoch_31_global_step_8000.ckpt"
# 1 mug overfit inverse kl + no rotations during training
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-16_104118/residual_flow_occlusion/rox5pvz3/checkpoints/epoch_35_global_step_9000.ckpt"
# 2 mug overfit inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-13_140201/residual_flow_occlusion/2ofq050u/checkpoints/epoch_31_global_step_8000.ckpt"
# 3 mug overfit inverse kl
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-17_154329/residual_flow_occlusion/3cejcxfp/checkpoints/epoch_35_global_step_9000.ckpt"


# L2 normed thing but trained for much longer
# place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-19_165523/residual_flow_occlusion/vmd5lk3a/checkpoints/epoch_7_global_step_2000.ckpt"
# EVEN LONGER
place_checkpoint_file="/home/jwang/code/equivariant_pose_graph/logs/residual_flow_occlusion/2023-01-24_120334/residual_flow_occlusion/3k1tmmni/checkpoints/epoch_83_global_step_21000.ckpt"


inner_network = ResidualFlow_DiffEmbTransformer(
                    emb_nn='dgcnn', return_flow_component=True, center_feature=True,
                    inital_sampling_ratio=1, input_dims=4)
place_network = Multimodal_ResidualFlow_DiffEmbTransformer(
                    inner_network, gumbel_temp=0.5, center_feature=True)
place_model = EquivarianceTrainingModule(
    place_network,
    lr=1e-4,
    image_log_period=100,
    weight_normalize='softmax', #'l1',
    softmax_temperature=1
)

# inner_network = ResidualFlow_DiffEmbTransformer(
#                     emb_nn='dgcnn', return_flow_component=True, center_feature=True,
#                     inital_sampling_ratio=1, input_dims=4)
# place_nocond_network = Multimodal_ResidualFlow_DiffEmbTransformer(
#                     inner_network, gumbel_temp=0.5, center_feature=True)
# place_network = Multimodal_ResidualFlow_DiffEmbTransformer_WithPZCondX(
#                     place_nocond_network, encoder_type="2_dgcnn")
# place_nocond_model = EquivarianceTrainingModule(
#     place_nocond_network,
#     lr=1e-4,
#     image_log_period=100,
#     weight_normalize='softmax', #'l1',
#     softmax_temperature=1
# )
# place_model = EquivarianceTrainingModule_WithPZCondX(
#     place_network,
#     place_nocond_model,
# )


place_model.cuda()
place_model.load_state_dict(torch.load(place_checkpoint_file)['state_dict']);

In [28]:
def vis(MUG_ID):
    from torch.utils.data import DataLoader

    #####

    fig2 = go.FigureWidget(
        make_subplots(
                column_widths=[0.5, 0.5],
                row_heights=[1, 1],
                rows=2, cols=2,
                specs=[[{'type': 'surface'}, {'type': 'surface'}], [{'type': 'surface'}, {'type': 'surface'}]],
#                 subplot_titles=([f"Mug on left rack", f"Mug on right rack", f"Rack with mug on left", f"Rack with mug on right"]),
                subplot_titles=([f"Mug #999{MUG_ID}'s p(z|Y) (left)", f"Mug #{MUG_ID}'s p(z|Y) (right)", f"Rack's p(z|Y) (left)", f"Rack's p(z|Y) (right)"]),
        )
    )
    
    fig2.update_layout(
        height=800,
    )
    
    for i, mug_id in enumerate(["999"+MUG_ID, MUG_ID]):

        # dataset = PointCloudDataset('/home/bokorn/src/ndf_robot/data/renders', 
        # dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/bokorn_ndf_robot_data_duprack_bothmugrack/renders', 
        dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders', 
#         dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/test_data_duprack_bothmugrack/renders', 
        # dataset = PointCloudDataset('/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack_overfit/renders', 
                                    cloud_type='teleport',
        #                             cloud_type='init',
                                    dataset_indices=[mug_id])
        #                             dataset_indices=[3])
        dataloader = DataLoader(dataset)

        ####

        from pytorch3d.transforms import Transform3d
        from pytorch_lightning.utilities.seed import seed_everything

        seed_everything(0)

        for data in dataloader:
            points_action = data['points_action'].cuda()[:,:-2]
            points_anchor = data['points_anchor'].cuda()
            points_action_trans = data['points_action_trans'].cuda()[:,:-2]
            points_anchor_trans = data['points_anchor_trans'].cuda()
        #     T0 = Transform3d(matrix=data['T0']).cuda()
        #     T1 = Transform3d(matrix=data['T1']).cuda()

            from equivariant_pose_graph.utils.se3 import random_se3
        #     T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
        #     T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T0 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            T1 = random_se3(1, rot_var=180, trans_var=0.5).cuda()
            points_action_trans = T0.transform_points(points_action)
            points_anchor_trans = T1.transform_points(points_anchor)
            break



        ######

        vis_train = True

        if vis_train:
            place_model.train()
            res = place_model.get_transform(points_action_trans, points_anchor_trans,  points_action, points_anchor)

            pred_T_action = res['pred_T_action']
            flow_action = res['flow_components']['flow_action'][:,:,:3]
            pred_w_action = torch.sigmoid(res['flow_components']['flow_action'][:,:,3])
            flow_anchor = res['flow_components']['flow_anchor'][:,:,:3]
            pred_w_anchor = torch.sigmoid(res['flow_components']['flow_anchor'][:,:,3])
            corr_flow_action = res['flow_components']['corr_flow_action']
            corr_flow_anchor = res['flow_components']['corr_flow_anchor']
            corr_points_action = res['flow_components']['corr_points_action']
            corr_points_anchor = res['flow_components']['corr_points_anchor']
            residual_flow_action = res['flow_components']['residual_flow_action']
            residual_flow_anchor = res['flow_components']['residual_flow_anchor']
            action_attn = res['flow_components']['action_attn']
            anchor_attn = res['flow_components']['anchor_attn']
            goal_emb_action = res['flow_components']['goal_emb'][:, :, :residual_flow_action.shape[1]]
            goal_emb_anchor = res['flow_components']['goal_emb'][:, :, residual_flow_action.shape[1]:]
            dense_trans_pt_action = res['flow_components']['dense_trans_pt_action']
            dense_trans_pt_anchor = res['flow_components']['dense_trans_pt_anchor']
            trans_pt_action = res['flow_components']['trans_pt_action']
            trans_pt_anchor = res['flow_components']['trans_pt_anchor']
            trans_sample_action = res['flow_components']['trans_sample_action']
            trans_sample_anchor = res['flow_components']['trans_sample_anchor']
#             goal_emb_cond_x_action = res['flow_components']['goal_emb_cond_x'][:, :, :residual_flow_action.shape[1]]
#             goal_emb_cond_x_anchor = res['flow_components']['goal_emb_cond_x'][:, :, residual_flow_action.shape[1]:]
        else:
            place_model.eval()
            res = place_model.training_module_no_cond_x.get_transform(points_action_trans, points_anchor_trans, points_action, points_anchor)
            goal_emb_action = res['flow_components']['goal_emb'][:, :, :residual_flow_action.shape[1]]
            goal_emb_anchor = res['flow_components']['goal_emb'][:, :, residual_flow_action.shape[1]:]

            res = place_model.get_transform(points_action_trans, points_anchor_trans, points_action, points_anchor)
            pred_T_action = res['pred_T_action']
            flow_action = res['flow_components']['flow_action'][:,:,:3]
            pred_w_action = torch.sigmoid(res['flow_components']['flow_action'][:,:,3])
            flow_anchor = res['flow_components']['flow_anchor'][:,:,:3]
            pred_w_anchor = torch.sigmoid(res['flow_components']['flow_anchor'][:,:,3])
            corr_flow_action = res['flow_components']['corr_flow_action']
            corr_flow_anchor = res['flow_components']['corr_flow_anchor']
            corr_points_action = res['flow_components']['corr_points_action']
            corr_points_anchor = res['flow_components']['corr_points_anchor']
            residual_flow_action = res['flow_components']['residual_flow_action']
            residual_flow_anchor = res['flow_components']['residual_flow_anchor']
            action_attn = res['flow_components']['action_attn']
            anchor_attn = res['flow_components']['anchor_attn']
            dense_trans_pt_action = res['flow_components']['dense_trans_pt_action']
            dense_trans_pt_anchor = res['flow_components']['dense_trans_pt_anchor']
            trans_pt_action = res['flow_components']['trans_pt_action']
            trans_pt_anchor = res['flow_components']['trans_pt_anchor']
            trans_sample_action = res['flow_components']['trans_sample_action']
            trans_sample_anchor = res['flow_components']['trans_sample_anchor']
#             goal_emb_cond_x_action = res['flow_components']['goal_emb_cond_x'][:, :, :residual_flow_action.shape[1]]
#             goal_emb_cond_x_anchor = res['flow_components']['goal_emb_cond_x'][:, :, residual_flow_action.shape[1]:]

        ####

        post_action_pts = toDisplay(pred_T_action.transform_points(points_action_trans))[0]
        post_anchor_pts = toDisplay(pred_T_action.inverse().transform_points(points_anchor_trans))[0]

        action_pts = toDisplay(points_action)[0]
        anchor_pts = toDisplay(points_anchor)[0]
        action_trans_pts = toDisplay(points_action_trans)[0]
        anchor_trans_pts = toDisplay(points_anchor_trans)[0]
        action_flow = toDisplay(flow_action)[0]
        action_flow_corr = toDisplay(corr_flow_action)[0]
        action_flow_res = toDisplay(residual_flow_action)[0]
        anchor_flow = toDisplay(flow_anchor)[0]
        # vis_scores = toDisplay(scores)[0]
        vis_attn_act = toDisplay(action_attn)[0]
        vis_attn_anc = toDisplay(anchor_attn)[0]
        action_w = toDisplay(pred_w_action)[0]
        anchor_w = toDisplay(pred_w_anchor)[0]

        action_pts_dmean = action_pts - np.concatenate([action_pts, anchor_pts], axis=0).mean(axis=0)
        fig2.add_trace(
            go.Scatter3d(
                x = action_pts_dmean[:,0], 
                y = action_pts_dmean[:,1], 
                z = action_pts_dmean[:,2], 
                mode = 'markers',
                hoverinfo = 'none',
                name='action_demo',
            ), 
            row=1, col=i+1
        )

        goal_emb_action_colors = color_gradient(F.softmax(goal_emb_action[0], dim=-1))
        fig2.data[2*i].marker.color = goal_emb_action_colors[0]
        
        anchor_pts_dmean = anchor_pts - np.concatenate([action_pts, anchor_pts], axis=0).mean(axis=0)
        fig2.add_trace(
            go.Scatter3d(
                x = anchor_pts_dmean[:,0], 
                y = anchor_pts_dmean[:,1], 
                z = anchor_pts_dmean[:,2], 
                mode = 'markers',
                hoverinfo = 'none',
                name='anchor_demo',
            ), 
            row=2, col=i+1
        )

        goal_emb_anchor_colors = color_gradient(F.softmax(goal_emb_anchor[0], dim=-1))
        fig2.data[2*i+1].marker.color = goal_emb_anchor_colors[0]
        
    del dataset
    del dataloader


    return fig2

In [55]:
#0, 1, 2, 3, 4, 6, 8, 10, 11, 14, 22, 28

vis("28")

Global seed set to 0


/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/99928_teleport_obj_points.npz


Global seed set to 0


hi
/home/jwang/code/equivariant_pose_graph/data/train_data_duprack_bothmugrack/renders/28_teleport_obj_points.npz
hi


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'color': array([[  0.        ,   0.…